In [1]:
import nltk.classify.util
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import names

import sys
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt

from sklearn.datasets import load_files
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn import metrics

from sklearn.cluster import KMeans

import logging
from optparse import OptionParser



from sklearn.cluster import KMeans
import nltk

import gensim

from gensim.models import CoherenceModel
from nltk.corpus import stopwords
import sqlite3 as lite

from sklearn.model_selection import train_test_split




In [2]:
p='../text_mining_class/MyCode/Training.txt'

file = open(p,"r")
headers=[]
tony_id=[]
sentiment=[]
text = []
i=0

for sent in file:
    if i < 1 :
        var = sent.split('\t')
        headers.append(var[0])
        headers.append(var[1])
        headers.append(var[2])
        i=i+1
    else:
        var = sent.split('\t')
        if len(var) >= 3:
            tony_id.append(var[0])
            sentiment.append(var[1])
            text.append(var[2])
            
text[0]


'@dog_crazy http://twitpic.com/5f4jq - Awww so Toro is your baby?? Soo sweet!\n'

In [3]:
# # Remove Emails
# text = [re.sub('\S*@\S*\s?', '', sent) for sent in text]

# # Remove new line characters
# text = [re.sub('\s+', ' ', sent) for sent in text]

# # Remove http
# #text = [re.sub('^http://\w', ' ', sent) for sent in text]

# # Remove distracting single quotes
# text = [re.sub("\'", "", sent) for sent in text]



In [4]:
print(text[:1])

['@dog_crazy http://twitpic.com/5f4jq - Awww so Toro is your baby?? Soo '
 'sweet!\n']


In [15]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

# data_words_nostops = remove_stopwords(text)


In [24]:

positive_sent =[]
negative_sent =[]

#split from: https://github.com/scikit-learn/scikit-learn/blob/7b136e9/sklearn/model_selection/_split.py#L2073

train, test = train_test_split(text,test_size = 0.1)

for i in range (len(train)):
    if sentiment[i] =='+':
        positive_sent.append(text[i])
    else:
        negative_sent.append(text[i])
        
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

# positive_sent = remove_stopwords(positive_sent)
# negative_sent = remove_stopwords(negative_sent)
        
        
positive_vocab = list(sent_to_words(positive_sent))
negative_vocab = list(sent_to_words(negative_sent))
      

In [25]:
test_positive_sent =[]
test_negative_sent =[]
test_data_positive_and_neg_binary =[]

for i in range (len(test)):
    if sentiment[i] =='+':
        test_positive_sent.append(text[i])
        test_data_positive_and_neg_binary.append(1)
    else:
        test_negative_sent.append(text[i])
        test_data_positive_and_neg_binary.append(0)
        
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

test_positive_vocab = list(sent_to_words(test_positive_sent))
test_negative_vocab = list(sent_to_words(test_negative_sent))
      

In [26]:

test_data_positive_and_neg = list(sent_to_words(test))

In [27]:
# from : https://streamhacker.com/2010/05/10/text-classification-sentiment-analysis-naive-bayes-classifier/

def word_feats(words):
    return dict([(word, True) for word in words])

positive_features = [(word_feats(pos), 'pos') for pos in positive_vocab]
negative_features = [(word_feats(neg), 'neg') for neg in negative_vocab]


train_set = negative_features + positive_features

print (train_set[:5])




[({'give': True, 'up': True}, 'neg'), ({'mrsangell': True, 'oh': True, 'no': True, 'truly': True, 'sorry': True, 'to': True, 'hear': True, 'that': True}, 'neg'), ({'gah': True, 'tired': True, 'and': True, 'my': True, 'eye': True, 'hurt': True}, 'neg'), ({'cus': True, 'that': True, 'man': True, 'sed': True, 'he': True, 'doesnt': True, 'want': True, 'me': True, 'to': True, 'work': True, 'there': True, 'im': True, 'lazy': True, 'and': True, 'dnt': True, 'no': True, 'anything': True}, 'neg'), ({'gotta': True, 'blow': True, 'out': True, 'then': True, 'flat': True, 'iron': True, 'my': True, 'hair': True, 'that': True, 'might': True, 'take': True, 'while': True}, 'neg')]


In [28]:
#train
classifier = NaiveBayesClassifier.train(train_set)

In [29]:
# Predict
neg = 0
pos = 0

result_test =[]
result_pos_test =[]

for word in test_data_positive_and_neg:
    classResult = classifier.classify( word_feats(word))
    if classResult == 'neg':
        neg = neg + 1
        result_test.append(0) 
    if classResult == 'pos':
        pos = pos + 1
        result_test.append(1) 
        

print('Positive: ' + str(float(pos)/len(test_data_positive_and_neg)))
print('Negative: ' + str(float(neg)/len(test_data_positive_and_neg)))


Positive: 0.4226142261422614
Negative: 0.5773857738577386


In [81]:
p='/Users/youssefkindo/Downloads/text_mining_class/MyCode/sentiment_testfiles/test1_public.txt'

file_1 = open(p,"r",encoding = "ISO-8859-1")
headers_1=[]
tony_id_1=[]
# sentiment=[]
text_1 = []
i=0

for sent in file_1:
    if i < 1 :
        var = sent.split('XXX')
        headers_1.append(var[0])
        headers_1.append(var[1])
        i=i+1
    else:
        var = sent.split('XXX')
        if len(var) >= 2:
            tony_id_1.append(var[0])
#             sentiment.append(var[1])
            text_1.append(var[1:])
            



In [77]:
p2='/Users/youssefkindo/Downloads/text_mining_class/MyCode/sentiment_testfiles/test2_public.txt'

file_2 = open(p2,"r",encoding = "ISO-8859-1")
headers_2=[]
tony_id_2=[]
# sentiment=[]
text_2 = []
i=0

for sent in file_2:
    if i < 1 :
        var = sent.split('XXX')
        headers_2.append(var[0])
        headers_2.append(var[1])
        i=i+1
    else:
        var = sent.split('XXX')
        if len(var) >= 2:
            tony_id_2.append(var[0])
#             sentiment.append(var[1])
            text_2.append(var[1:])
            



In [78]:
print(len(text_2))

100000


In [83]:
#prepare--
text_2_prepared = sent_to_words(text_2)

# Predict
neg = 0
pos = 0

result_test_2 =[]
result_pos_test =[]

for word in text_2_prepared:
    classResult = classifier.classify( word_feats(word))
    if classResult == 'neg':
        neg = neg + 1
        result_test_2.append(0) 
    if classResult == 'pos':
        pos = pos + 1
        result_test_2.append(1) 
        

print('Positive: ' + str(float(pos)/len(text_2)))
print('Negative: ' + str(float(neg)/len(text_2)))



Positive: 0.4118
Negative: 0.5882


In [88]:
fw5=open('/Users/youssefkindo/Downloads/text_mining_class/MyCode/sentiment_testfiles/result_2.txt','w')

fw5.write('ID#,' +"  "+'sentiment'+",\n")

for i in range(len(text_2)):
    #name = patent_id[labels[i]]
    #print(name)
    if result_test_2[i] == 1:
        fw5.write(str( tony_id_2[i]) +", "+"+"+",\n")
    else:
        fw5.write(str( tony_id_2[i]) +", "+"-"+",\n")
        
    
fw5.close()

In [86]:
#prepare--
text_1_prepared = sent_to_words(text_1)

# Predict
neg = 0
pos = 0

result_test_1 =[]

for word in text_1_prepared:
    classResult = classifier.classify( word_feats(word))
    if classResult == 'neg':
        neg = neg + 1
        result_test_1.append(0) 
    if classResult == 'pos':
        pos = pos + 1
        result_test_1.append(1) 
        

print('Positive: ' + str(float(pos)/len(text_1)))
print('Negative: ' + str(float(neg)/len(text_1)))


Positive: 0.4107741077410774
Negative: 0.5892258922589226


In [89]:
fw5=open('/Users/youssefkindo/Downloads/text_mining_class/MyCode/sentiment_testfiles/result_1.txt','w')

fw5.write('ID#,' +"  "+'sentiment'+",\n")

for i in range(len(text_1)):
    #name = patent_id[labels[i]]
    #print(name)
    if result_test_1[i] == 1:
        fw5.write(str( tony_id_1[i]) +", "+"+"+",\n")
    else:
        fw5.write(str( tony_id_1[i]) +", "+"-"+",\n")
        
    
fw5.close()

In [23]:
# print(len(test_data_positive_and_neg_binary))
# print(len(result_test))

# print(test_data_positive_and_neg_binary[990:1000])
# print(result_test[990:1000])

In [23]:
#from : https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html

# from sklearn.metrics import f1_score
# y_true = test_data_positive_and_neg_binary
# y_true =[]
# l=len(test_negative_vocab)
# for i in range (l):
#     y_true.append(0)

# g=len(test_positive_vocab)
# for i in range (g):
#     y_true.append(1)

    
# y_pred = result_test
# y_pred = []
# y=len(result_neg_test)
# for i in range (y):
#     y_pred.append(0)

# z=len(result_pos_test)
# for i in range (z):
#     y_pred.append(1)

# k=393
# for i in range (k):
#     y_pred.append(2)

# score = f1_score(test_negative_vocab_w_feat,neg)

In [1]:

# f1_score(y_true, y_pred)  


In [26]:
fw5=open('/Users/youssefkindo/Downloads/text_mining_class/contest_result.txt','w')

nameClustDict = {}
nameDict = {}
for i in range(len(test_negative_vocab)):
    #name = patent_id[labels[i]]
    #print(name)
    if i == 0:  
        fw5.write('id,' +"  "+'snetiment'+",\n")
    
    fw5.write(str( tony_id[i]) +", "+str(result_test[i])+",\n")
    
fw5.close()

